In [ ]:
import pandas as pd
import numpy as np
import python_functions_sp as pfsp
import warnings
warnings.filterwarnings("ignore")
import sqlalchemy
import psycopg2
import sql_functions_sp as sfsp

In [ ]:
schema = 'capstone_health_education'

hepa_continents = sfsp.get_dataframe(f'SELECT * FROM {schema}.hepa_filtered_final')

In [ ]:
hepa_continents

Changing the country-code to the country-name:

In [ ]:
hepa_continents.rename(columns={'country': 'country_code'}, inplace=True)
hepa_continents

Add a column with the country name by using the convert-method in country_converter:

In [ ]:
import country_converter as coco

hepa_continents['country_name'] = hepa_continents['country_code'].apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))

Checking if it worked:

In [ ]:
unique_country_codes = hepa_continents['country_code'].unique()
unique_country_names = hepa_continents['country_name'].unique()

print("unique country-codes:")
print(unique_country_codes)

print("\nunique country-names:")
print(unique_country_names)


Delete unneeded column and rename:

In [ ]:
hepa_continents.drop('country_code', axis=1, inplace=True)

In [ ]:
hepa_continents.rename(columns={'country_name': 'country'}, inplace=True)

In [ ]:
hepa_continents.sample(10)

## Splitting the countries into the continents
- northern america
- southern america
- europe (4 parts for further analysis)
- africa
- asia
- pacific
- middle-east

Assigning the continent to the countries in a new created column:

In [ ]:
hepa_continents['continent'] = hepa_continents['country'].apply(pfsp.assign_continent)

Assigning the four parts of europe to the countries in another new created column:

In [ ]:
hepa_continents['continent_region'] = hepa_continents['country'].apply(pfsp.assign_europe_region)

In [ ]:
hepa_continents.columns

Changing the order of the columns:

In [ ]:
columns = ['measure code', 'country', 'continent', 'continent_region', 'year', 'value']

hepa_continents = hepa_continents[columns]

Checking the result:

In [ ]:
hepa_continents.sample(10)

---

Check if some countries couldn't be assigned to a continent:

In [ ]:
hepa_continents[hepa_continents['continent'] == 'Other']['country'].unique()

In [ ]:
hepa_continents[hepa_continents['continent_region'] == 'Other']['country'].unique()

---

Check if the new created columns match:

In [ ]:
pfsp.check_values(hepa_continents)

---

# Writing the table into the database

In [ ]:
# sfsp.push_to_cloud(hepa_continents, 'hepa_continents')

In [ ]:
hepa_continents = hepa_continents

%store hepa_continents